In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Aug  9 12:46:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [4]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
#!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/requirements.txt

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-itaomm38
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-itaomm38
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [5]:
import sys
from collections import defaultdict 
from collections import defaultdict 
from transformers import BertForMaskedLM, BertModel, BertTokenizer, BertForSequenceClassification

sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/')
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [6]:
#adapter_list = ["sst"]
#for adapter in adapter_list:
#  print("!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1")

## Sentiment Bert


In [7]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/')
except OSError as e:
        pass

In [8]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sentiment_bert/SentimentBert.pth --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sentiment_bert/SentimentBert.pth
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selec

In [9]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 46.77033240511501
		SS Score: 48.865221252177776
		ICAT Score: 45.7088528202769
	profession
		Count: 2430.0
		LM Score: 54.00482247355584
		SS Score: 49.334996261109005
		ICAT Score: 53.28655429629466
	race
		Count: 2886.0
		LM Score: 54.46395868877134
		SS Score: 38.71398348752011
		ICAT Score: 42.17033594684142
	religion
		Count: 237.0
		LM Score: 47.81609195402299
		SS Score: 37.37931034482759
		ICAT Score: 35.74665081252477
	overall
		Count: 2106.0
		LM Score: 53.06327578374257
		SS Score: 43.98156249291462
		ICAT Score: 46.67611559922874
intersentence
	gender
		Count: 726.0
		LM Score: 89.85632222045265
		SS Score: 61.56639857726814
		ICAT Score: 69.07004147066883
	profession
		Count: 2481.0
		LM Score: 86.11345278610756
		SS Score: 59.76348576305937
		ICAT Score: 69.29810338040663
	race
		Count: 2928.0
		LM Score: 87.11458859384024
		SS Score: 57.47634577825531
		ICAT Score: 74.08861286068013
	religion
		Count: 234.0
		LM Score: 91

# Adapter Fusion


In [13]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-sst-imdb/')
except OSError as e:
        pass

In [15]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/pytorch_model_adapter_fusion.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/pytorch_model_adapter_fusion.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-sst-imdb/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-sst-imdb/predictions.json


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/pytorch_model_adapter_fusion.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strate

In [16]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-sst-imdb/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-sst-imdb/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 26.13426082773909
		SS Score: 42.58381489685837
		ICAT Score: 22.25793051109316
	profession
		Count: 2430.0
		LM Score: 24.7810561028714
		SS Score: 46.818668285512075
		ICAT Score: 23.204320908900016
	race
		Count: 2886.0
		LM Score: 19.71821666181442
		SS Score: 51.174393657392756
		ICAT Score: 19.255077690159798
	religion
		Count: 237.0
		LM Score: 25.26436781609195
		SS Score: 58.62068965517241
		ICAT Score: 20.908442330558856
	overall
		Count: 2106.0
		LM Score: 22.66358474224213
		SS Score: 48.71567643247598
		ICAT Score: 22.081437222061343
intersentence
	gender
		Count: 726.0
		LM Score: 89.27522960131655
		SS Score: 57.683415618198225
		ICAT Score: 75.5564557325768
	profession
		Count: 2481.0
		LM Score: 82.14922838803075
		SS Score: 62.02679214659193
		ICAT Score: 62.389394491515645
	race
		Count: 2928.0
		LM Score: 85.85695281724048
		SS Score: 60.667684364322604
		ICAT Score: 67.53905535450329
	religion
		Count: 234.0
		LM Sco

Fusion pre-trained and fine tuned

In [17]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-pre-fine/')
except OSError as e:
        pass

In [18]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion3/pytorch_model_adapter_fusion.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion3/pytorch_model_adapter_fusion.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-pre-fine/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-pre-fine/predictions.json


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion3/pytorch_model_adapter_fusion.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strate

In [19]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-pre-fine/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/fusion-pre-fine/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 57.22502642285251
		SS Score: 50.4376454463411
		ICAT Score: 56.724140978238296
	profession
		Count: 2430.0
		LM Score: 61.59333662616648
		SS Score: 51.24620023667777
		ICAT Score: 60.05818401254043
	race
		Count: 2886.0
		LM Score: 62.71292921930271
		SS Score: 59.53688167137289
		ICAT Score: 50.75121351470924
	religion
		Count: 237.0
		LM Score: 62.22988505747126
		SS Score: 50.98850574712643
		ICAT Score: 60.999593077024706
	overall
		Count: 2106.0
		LM Score: 61.574752785833304
		SS Score: 54.91209770853859
		ICAT Score: 55.52552874457087
intersentence
	gender
		Count: 726.0
		LM Score: 10.150291536161102
		SS Score: 39.927182318486665
		ICAT Score: 8.105450815001928
	profession
		Count: 2481.0
		LM Score: 18.652360200395055
		SS Score: 38.71645915298121
		ICAT Score: 14.44306683610575
	race
		Count: 2928.0
		LM Score: 15.029834831327232
		SS Score: 38.63747938457492
		ICAT Score: 11.614298668979439
	religion
		Count: 234.0
		LM Sco

single imdb pretrained adapter


In [20]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/imdb-pretrained/')
except OSError as e:
        pass

In [21]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/imdb-pretrained/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/imdb-pretrained/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/imdb-pretrained/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/imdb-pretrained/predictions.json


Loading /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json...
---------------------------------------------------------------
                     ARGUMENTS                 
Pretrained Model: /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/imdb-pretrained/pytorch_adapter.bin
Skip Intrasentence: False
Skip Intersentence: False
Batch Size: 1
Max Seq Length: None
CUDA: True
---------------------------------------------------------------

Evaluating bias on intersentence tasks...
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy se

In [22]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/imdb-pretrained/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/imdb-pretrained/summary.json

intrasentence
	gender
		Count: 765.0
		LM Score: 57.21176005958614
		SS Score: 50.82091338613078
		ICAT Score: 56.272442066045805
	profession
		Count: 2430.0
		LM Score: 59.30420153003146
		SS Score: 51.455441935838834
		ICAT Score: 57.57792509246656
	race
		Count: 2886.0
		LM Score: 62.42487605777936
		SS Score: 42.010674150203855
		ICAT Score: 52.45022253860461
	religion
		Count: 237.0
		LM Score: 58.71264367816092
		SS Score: 39.67816091954023
		ICAT Score: 46.59219447747392
	overall
		Count: 2106.0
		LM Score: 60.43895083052336
		SS Score: 46.623938532942184
		ICAT Score: 56.35803857035672
intersentence
	gender
		Count: 726.0
		LM Score: 69.5604174408522
		SS Score: 49.86717187804144
		ICAT Score: 69.37562584862576
	profession
		Count: 2481.0
		LM Score: 67.70305501293636
		SS Score: 55.71365404120318
		ICAT Score: 59.96641833540706
	race
		Count: 2928.0
		LM Score: 65.85713535258553
		SS Score: 52.38923356748718
		ICAT Score: 62.710173783726646
	religion
		Count: 234.0
		LM Score:

sst pretrained adapter

## SST Adapter


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_model.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/summary.json

# RoBERTa


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_model_head.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/roberta/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/predictions.json


In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/roberta/summary.json

#Language Poem (pre-trained)


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/lmpoem/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/lmpoem/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/lmpoem/summary.json

#Common Sense Winogrande (pre-trained)


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/winogrande/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/winogrande/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/predictions.json


In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/winogrande/summary.json

#SST Pretrained


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst_pretrained/pytorch_adapter.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst_pretrained/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/predictions.json


In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst-pretrained/summary.json